In [ ]:
### Author : Karssien Hero Huisman
### Hartree Fock Approximation : Semi Infinite Leads
### Onsite energy of the leads shifts with voltage:
### epsilon0L,epsilon0R -> ef + V/2, ef - V/2

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.insert(0, '/Users/khhuisman/Documents/Jupyter_notebooks/Paper3_SIFLeads/Modules_SIF')

### Scattering Region

In [3]:
import Sgeom_scatteringregion #Hamiltonian without interactions U=0, not coupled to leads

### Import NEGF methods

In [4]:
import negf_SIF_HFA as negf_method               ### negf code
import Integration_SIF_HFA_OnSiteVoltageDep as Integration_method ### for calculating electron densities
import Currents_HFA_SIF as Current_method        ### for calculatin currents

# P Value

In [5]:
import handy_functions_coulomb as hfc #some "usefull functions"
import Pvalue ## for calculating the P-value

In [6]:



def func_energies_voltagedepleads(Hamiltonian0,U,npoints,ef,Vmax,
                                 tleadL,tleadR,pz):
    

    #If tleadL or tleadR is enormous treat it like WBL
    evlist = np.linalg.eigh(Hamiltonian0)[0]
    evlist_shifted = np.add(evlist,[U/2 for i in range(len(evlist))])
    e_lumo = evlist_shifted[int(Hamiltonian0.shape[0]/2)-1]
    e_homo = evlist_shifted[int(Hamiltonian0.shape[0]/2)]
    
    #Fermi Energy
    hl_gap = e_lumo - e_homo
    
    
    
    #lower,upper bound for Glesser function
    emin = np.round(int(10*min(evlist_shifted))/10 - 10,2) #lower bound for integrals
    emax = np.round(int(10*max(evlist_shifted))/10 + 10,2)   #lower bound for integrals
    
    epsilon0_list = [ef + Vmax/2,ef - Vmax/2]
    energieslist = []
    #####
    de = 0.1
    emin_list = []
    emax_list = []
    
    for epsilon0L in epsilon0_list:
        for epsilon0R in epsilon0_list:
            
            
            
            eminL = -2*tleadL + epsilon0L - abs(pz)*2*tleadL - de  
            emaxL = epsilon0L +  abs(pz)*2*tleadL + 2*tleadL + de  

            eminR = epsilon0R - 2*tleadR  - de 
            emaxR = epsilon0R + 2*tleadR  + de  
    
    
            emin_list.append(eminL)
            emin_list.append(eminR)
            emax_list.append(emaxL)
            emax_list.append(emaxR)
    
  
    energies = np.linspace(min(emin_list),max(emax_list),npoints)
    
    return energies






In [7]:
def plot_occupation_difference(nP_list_conv,nM_list_conv,V_list_convg,Hamiltonian0):
    dimlist = hfc.func_list_i(Hamiltonian0)
    plt.title('$n_{is}(m,V) - n_{i\overline{s}}(-m,V)$')
    n_list_total_convgM_swap = [hfc.pairwise_swap([ nM_list_conv[i][k] for k in dimlist]) for i in range(len(V_list_convg))
                              ]
    nP_list_plot =[ [nP_list_conv[i][k] for k in dimlist ] for i in range(len(V_list_convg)) ]


    plt.plot(V_list_convg,np.subtract(nP_list_plot,n_list_total_convgM_swap))
    plt.xlabel('Bias Voltage [eV] ')
    plt.ylabel('Electron Density')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()


def plot_densities_symmtry_relation(dim,V_list,n_list):
    ones = [1 for i in range(dim)]
    n_list_swap = [np.subtract(ones,hfc.pairwise_swap(n_list[-1-i])) for i in range(len(n_list))]

    plt.title('$n_{is}(m,V) + n_{i\overline{s}}(m,-V) - 1$')
    plt.plot(V_list,np.subtract(n_list,n_list_swap))
    plt.xlabel('Bias Voltage [eV] ')
    plt.ylabel('Electron Density')
    plt.ticklabel_format(style="sci", scilimits=(0,0))


    plt.show()

# Scattering Region parameters

In [8]:
#Geometric Paramters
Lm      = 2            # "Length" of S-shape
Wg      = 2            # Width of S-shape
epsilon = 0            # onsite energy
t       = 2.4          # Hopping paramter

Ulist  = [0.5*t,  t]              ## list of Coulomb interaction values
lambdalist   = [(1*(10**-1))*t] ## list of soc coupling values

T = 300   #temperature (Kelvin) of leads
betaL,betaR = negf_method.func_beta(T),negf_method.func_beta(T) ### beta = 1/(kBT) in eV



### Bias window

In [9]:
Vmax = 1 # Maximum bias voltage [eV]
dV = 0.5 # stepsize

V_list_pos_bias,V_list_total = hfc.func_V_total(Vmax,dV)
print(len(V_list_total),V_list_pos_bias)

5 [0.5 1. ]


#### Integration & Self-consistency parameters

In [10]:
tol            = 10**-4  # tolerance of for converence of electron densities
tol_nintegrand = 10**-15 # cut-off value that much be satisfied for lower,upper bound Glesser integral.

max_iteration  = 200     # max number of iterations
npoints        = 15000    # max number of points to integrate over
alpha          = 0.4    # linear mixing paramter
ef = 0 
npoints_current = 10000

In [11]:
tsom = lambdalist[0] ## spin-orbit coupling parameter.
factor1 = 1 
factor2 = 1


# 3. Electron Densities

Note: Hubbard One converges well for large U 

In [ ]:
parameterlist = []
for U in Ulist:

    #Coupling lead-molecule
    pz             = 0.8               ### magnetic polarization of left lead.
    tlead          = (3)*t             ### hopping parameter lead strength
    tleadL,tleadR  = np.round(factor1*tlead,2),np.round(tlead,2)      # hopping parameter left,right lead
    tcoupL,tcoupR  =  factor2*np.sqrt(tleadL/factor1)*0.5 ,np.sqrt(tleadR)*0.5 #coupling of right lead to the molecule
    print('tleadL, tleadR = {},{} & tcoupLR = {},{}'.format(tleadL,tleadR,tcoupL,tcoupR))

    #### chi1,chi2
    chi1 = np.round(hfc.func_chi(tleadL , tleadR ),2)
    chi2 = np.round(hfc.func_chi(tcoupL , tcoupR ),2) 
    print(chi1,chi2)


    ### Scattering region
    Hamiltonian0,matL,matR = Sgeom_scatteringregion.hamiltonian_multiplesites_coupled_semi_inf(Lm,Wg, t,tsom,plotbool=False)



    print('--- ef = {}, tsom/t = {} , U/t = {},'.format(ef,tsom/t,U/t))
    n_list_totalP,convglistP = Integration_method.self_consistent_trapz_PN_OnSiteVoltDep_energyshift(
                            V_list_pos_bias,Vmax,
                              max_iteration,
                            ef,
                            Hamiltonian0,U,
                            tleadL,tcoupL,matL,
                            tleadR,tcoupR,matR,
                               abs(pz), 
                            betaL, betaR,tol,
                            tol_nintegrand,alpha,npoints,plotbool=False,trackbool=False)


    n_list_totalM,convglistM = Integration_method.self_consistent_trapz_PN_OnSiteVoltDep_energyshift(
                                V_list_pos_bias,Vmax,
                              max_iteration,
                            ef,
                            Hamiltonian0,U,
                            tleadL,tcoupL,matL,
                            tleadR,tcoupR,matR,
                               -abs(pz), 
                            betaL, betaR,tol,
                            tol_nintegrand,alpha,npoints,plotbool=False,trackbool=False)



    V_list_convg,nP_list_convg,nM_list_convg =  hfc.func_symmetric_converged(V_list_total,
                                                                  n_list_totalP ,convglistP,
                                                                  n_list_totalM, convglistM) 


    parameterlist.append([npoints_current, Lm,Wg,pz,tleadL,tleadR,tcoupL,tcoupR, T,alpha,t,U,tsom,ef,tol,Vmax,len(V_list_total),emin,emax,npoints,tol_nintegrand])




    plot_densities_symmtry_relation(Hamiltonian0.shape[0],V_list_convg,nP_list_convg)
    plot_densities_symmtry_relation(Hamiltonian0.shape[0],V_list_convg,nM_list_convg)





    print('calculating currents ...')
    IP_list = Current_method.calc_I_trapz_VoltDepLeads(npoints_current,
                                                        V_list_convg,ef,
                                                      Hamiltonian0,
                                                      tleadL,tcoupL,matL,
                                                    tleadR,tcoupR,matR,
                                                      abs(pz),
                                                      U,nP_list_convg,
                                                      betaL,betaR)
    print('+M done, now -M ...')
    IM_list = Current_method.calc_I_trapz_VoltDepLeads(npoints_current,
                                                        V_list_convg,ef,
                                                      Hamiltonian0,
                                                      tleadL,tcoupL,matL,
                                                     tleadR,tcoupR,matR,
                                                      -abs(pz),
                                                      U,nM_list_convg,
                                                      betaL,betaR)

    

    


    dIlist = np.subtract(IP_list,IM_list)
    Vprime, PClist = Current_method.func_MR_list(IP_list,IM_list,V_list_convg)

    plt.title('$U/t = {}$, $\lambda /t = {}$'.format(U/t,tsom/t))
    plt.plot(V_list_convg,IP_list)
    plt.plot(V_list_convg,IM_list)
    plt.xlabel('Bias Voltage [eV]')
    plt.ylabel('Current [eV]')
    plt.ticklabel_format(style="sci", scilimits=(0,0))
    plt.show()

    plt.title('Colinear $\Delta I(m): ef = {},U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,tsom/t))
    plt.plot(V_list_convg,dIlist)
    plt.xlabel('Bias Voltage [eV]')
    plt.ylabel('Current [eV]')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()

    plt.title('P-Value : $E_F = {} $, $U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,tsom/t))
    Vlist_prime,PJ_list = Pvalue.function_PvaluedI(V_list_convg,dIlist,22)
    plt.plot(Vlist_prime,PJ_list)
    plt.xlim(0,Vmax)
    plt.ylim(-1-0.1,1+0.1)
    plt.show()







        


tleadL, tleadR = 7.2,7.2 & tcoupLR = 1.3416407864998738,1.3416407864998738
0.0 0.0
--- ef = 0, tsom/t = 0.1 , U/t = 0.5,
--- V = 0 ---
[0.0092 0.0092 0.0108 0.0108 0.0025 0.0025 0.003  0.003  0.0008 0.0008
 0.0072 0.0072 0.0045 0.0045 0.0024 0.0024 0.0039 0.0039 0.0053 0.0053]
[0.0045 0.0045 0.0054 0.0054 0.001  0.001  0.0014 0.0014 0.0006 0.0006
 0.0036 0.0036 0.0021 0.0021 0.0012 0.0012 0.002  0.002  0.0026 0.0026]


In [ ]:
print(parameterlist)